In [1]:
import torch
torch.__version__
torch.cuda.is_available()



False

In [ ]:
tensor0d = torch.tensor(1)
tensor1d = torch.tensor([1,2,3,4])
tensor2d = torch.tensor([[1,2],[3,4]])
tensor3d = torch.tensor([
                        [[1, 2],[3, 4]],
                        [[5, 6], [7, 8]]
                        ])
print(tensor3d[1][1] [0])
print(tensor3d.shape)

tensor1dre = (tensor1d.reshape(4,1))

print(tensor2d.shape)
tensor2d @ tensor2d


tensor(7)
torch.Size([2, 2, 2])
torch.Size([2, 2])


tensor([[ 7, 10],
        [15, 22]])

In [17]:
import torch.nn.functional as F
from torch.autograd import grad

y = torch.tensor([1.0])
x1 = torch.tensor([1.1])
w1 = torch.tensor([3.2], requires_grad=True)
b = torch.tensor([2.0], requires_grad= True)

z = x1 * w1 + b
a = torch.sigmoid(z)

loss = F.binary_cross_entropy(a,y)

# grad_L_w1 = grad(loss, w1, retain_graph=True)
# grad_L_b = grad(loss, b, retain_graph=True)
# print(loss)
# print(grad_L_w1)
# print(grad_L_b)

loss.backward()
print(w1.grad)
print(b.grad)




tensor([-0.0044])
tensor([-0.0040])


In [ ]:
class NeuralNetwork(torch.nn.Module):
    def __init__(self, num_inputs, num_outputs):
        super().__init__()
        
        self.layers = torch.nn.Sequential(
            # 1st hidden layer
            torch.nn.Linear(num_inputs, 30), torch.nn.ReLU(),

            # 2nd hidden layer
            torch.nn.Linear(30, 20), torch.nn.ReLU(),

            # output layer
            torch.nn.Linear(20, num_outputs),
        )

    def forward(self, x):
        logits = self.layers(x)
        return logits

        


In [46]:
model = NeuralNetwork(50, 3)
print(model)
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print("Total number of trainable model parameters: ", num_params)

NeuralNetwork(
  (layers): Sequential(
    (0): Linear(in_features=50, out_features=30, bias=True)
    (1): ReLU()
    (2): Linear(in_features=30, out_features=20, bias=True)
    (3): ReLU()
    (4): Linear(in_features=20, out_features=3, bias=True)
  )
)
Total number of trainable model parameters:  2213


In [48]:
print(model.layers[0].weight)
print(model.layers[0].weight.shape)
print(model.layers[0].bias.shape)



Parameter containing:
tensor([[-0.0487,  0.1108,  0.0850,  ..., -0.0224,  0.1168, -0.0777],
        [ 0.0284, -0.0451, -0.0476,  ..., -0.1018, -0.0754, -0.0250],
        [-0.0073, -0.0393, -0.0651,  ...,  0.0613,  0.0995, -0.0556],
        ...,
        [ 0.0039,  0.0834, -0.0209,  ...,  0.0370,  0.0725, -0.1128],
        [ 0.0168, -0.0579, -0.0317,  ..., -0.1054,  0.0309,  0.0602],
        [-0.1262, -0.0735,  0.0196,  ..., -0.0298, -0.1090, -0.0806]],
       requires_grad=True)
torch.Size([30, 50])
torch.Size([30])


In [ ]:
torch.manual_seed(123)
model = NeuralNetwork(50,3)
print(model.layers[0].weight)


Parameter containing:
tensor([[-0.0577,  0.0047, -0.0702,  ...,  0.0222,  0.1260,  0.0865],
        [ 0.0502,  0.0307,  0.0333,  ...,  0.0951,  0.1134, -0.0297],
        [ 0.1077, -0.1108,  0.0122,  ...,  0.0108, -0.1049, -0.1063],
        ...,
        [-0.0787,  0.1259,  0.0803,  ...,  0.1218,  0.1303, -0.1351],
        [ 0.1359,  0.0175, -0.0673,  ...,  0.0674,  0.0676,  0.1058],
        [ 0.0790,  0.1343, -0.0293,  ...,  0.0344, -0.0971, -0.0509]],
       requires_grad=True)


In [90]:
X = torch.rand((1,50))
out = model(X)
print(out)

tensor([[-0.1139,  0.1325, -0.0884]], grad_fn=<AddmmBackward0>)


In [92]:
with torch.no_grad():
    out = model(X)
print(out)

tensor([[-0.1139,  0.1325, -0.0884]])


In [93]:
with torch.no_grad():
    out = torch.softmax(model(X), dim=1)
print(out)

tensor([[0.3026, 0.3871, 0.3104]])


In [96]:
X_train = torch.tensor([
         [-1.2, 3.1],
         [-0.9, 2.9],
         [-0.5, 2.6],
         [2.3, -1.1],
         [2.7, -1.5]
    ])

y_train = torch.tensor([0, 0, 0, 1, 1,])

X_test = torch.tensor([
        [-0.8, 2.8],
        [2.6, -1.6],
])

y_test = torch.tensor([0, 1])

In [97]:
# Defining a custom Dataset class
from torch.utils.data import Dataset

class ToyDataset(Dataset):
    def __init__(self, X, y):
        self.features = X
        self.labels = y
    
    def __getitem__(self, index):
        one_x = self.features[index]
        one_y = self.labels[index]
        return one_x, one_y

    def __len__(self):
        return self.labels.shape[0]

train_ds = ToyDataset(X_train, y_train)
test_ds = ToyDataset(X_test, y_test)


In [ ]:
from torch.utils.data import DataLoader

torch.manual_seed(123)

train_loader = DataLoader(
    dataset=train_ds,
    batch_size=2,
    shuffle=True,
    num_workers=0,
    drop_last=True
)

test_loader = DataLoader(
    dataset=test_ds,
    batch_size=2,
    shuffle=False,
    num_workers=0,
    drop_last=True
)

In [ ]:
# Iterate over the Dataloaders
for idx, (x, y) in enumerate(train_loader):
    print( f'Batch {idx+1}: ', x, y)


Batch 1:  tensor([[-1.2000,  3.1000],
        [-0.5000,  2.6000]]) tensor([0, 0])
Batch 2:  tensor([[-0.9000,  2.9000],
        [ 2.7000, -1.5000]]) tensor([0, 1])
Batch 3:  tensor([[ 2.3000, -1.1000]]) tensor([1])
tensor([[ 2.3000, -1.1000]]) tensor([1])


In [120]:
# Neural Network Training in PyTorch
import torch.nn.functional as F

torch.manual_seed(123)
model = NeuralNetwork(num_inputs=2, num_outputs=2)
optimizer = torch.optim.SGD(
    model.parameters(), lr=0.5
)

num_epochs = 3
for epoch in range(num_epochs):
    model.train()
    for batch_idx, (features, labels) in enumerate(train_loader):
        logits = model(features)

        loss = F.cross_entropy(logits, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        ### LOGGING
        print(f'Epoch: {epoch+1:03d}/{num_epochs:03d}'
        f' | Batch {batch_idx:03d}/{len(train_loader):03d}'
        f' | Train Loss: {loss:.2f}')
    model.eval()

Epoch: 001/003 | Batch 000/003 | Train Loss: 0.75
Epoch: 001/003 | Batch 001/003 | Train Loss: 0.65
Epoch: 001/003 | Batch 002/003 | Train Loss: 0.42
Epoch: 002/003 | Batch 000/003 | Train Loss: 0.05
Epoch: 002/003 | Batch 001/003 | Train Loss: 0.13
Epoch: 002/003 | Batch 002/003 | Train Loss: 0.00
Epoch: 003/003 | Batch 000/003 | Train Loss: 0.01
Epoch: 003/003 | Batch 001/003 | Train Loss: 0.00
Epoch: 003/003 | Batch 002/003 | Train Loss: 0.02


In [121]:
model.eval()
with torch.no_grad():
    outputs = model(X_train)
print(outputs)

tensor([[ 2.9320, -4.2563],
        [ 2.6045, -3.8389],
        [ 2.1484, -3.2514],
        [-2.1461,  2.1496],
        [-2.5004,  2.5210]])


In [122]:
# Softmaxing the stuff
torch.set_printoptions(sci_mode=False)
probas = torch.softmax(outputs, dim=1)
print(probas)

tensor([[    0.9992,     0.0008],
        [    0.9984,     0.0016],
        [    0.9955,     0.0045],
        [    0.0134,     0.9866],
        [    0.0066,     0.9934]])


In [124]:
# Convert probabilities in into class label predicitions 

predictions = torch.argmax(probas, dim=1)
print(predictions)

tensor([0, 0, 0, 1, 1])


In [ ]:
predictions == y_train
torch.sum(predictions==y_train)

tensor(5)

In [138]:
# A functionj to compute the prediction accuracy

def compute_accuracy(model, dataloader):
    model = model.eval()
    correct = 0.0
    total_examples = 0

    for idx, (features, labels) in enumerate(dataloader):
        with torch.no_grad():
            logits = model(features)

        predictions = torch.argmax(logits, dim=1)
        compare = labels == predictions
        correct += torch.sum(compare)
        total_examples += len(compare)
    
    return (correct /total_examples).item()

In [140]:
print(compute_accuracy(model, train_loader))


1.0


In [ ]:
# Save and load models for later
torch.save(model.state_dict(), "model.pth")